# Persistence and Streaming

Based on [**this tutorial**](https://learn.deeplearning.ai/courses/ai-agents-in-langgraph/lesson/5/persistence-and-streaming)

# Setup

In [ ]:
from dotenv import load_dotenv

In [ ]:
_ = load_dotenv()

# Imports and Basic Implementation

In [1]:
import operator
from typing import Annotated, TypedDict

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import AnyMessage, HumanMessage, SystemMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END

In [2]:
# Instanciate Search Tool
tool = TavilySearchResults(max_results=2)

In [3]:
# Define AgentState
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

# Persistence

In order to deal with **persistence**, we will use what's called a **checkpointer** into LangGraph.

It basically **checkpoints the state between every node**.

Here, we'll make use of a `SqliteSaver`, and use it "in memory"

In [4]:
from langgraph.checkpoint.sqlite import SqliteSaver

In [5]:
memory = SqliteSaver.from_conn_string(":memory:")

It's then really easy to incorporate it within our `Agent` class.

In [6]:
# Define Agent
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):  # 👈 REFERENCE TO CHECKPOINTER
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)  # 👈 REFERENCE TO CHECKPOINTER
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

> **NOTE**
>
> For data persistence, it can also be made use of other databases, or [**Redis**](https://redis.io/) for example.

In [7]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-3.5-turbo")  # 👈 Change to -4o for POC
abot = Agent(model, [tool], system=prompt, checkpointer=memory)